<a href="https://colab.research.google.com/github/ajinvinodk/CPP-learning/blob/main/solvathon_demo_only_1_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Liberaries to run GPT4ALL ####
!pip install gpt4all
!apt install -y libnvidia-gl-525-server
!apt install -y libvulkan1


### Liberaries to run text-to-Audio ####
! pip install transformers
! pip install datasets
! pip install sentencepiece
# !pip install pyttsx3
!apt install -y espeak

## Speech-2-Text
!pip install -q git+https://github.com/openai/whisper.git
!git clone -q  https://github.com/petewarden/openai-whisper-webapp

## web app
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libnvidia-common-525-server libnvidia-egl-wayland1
The following NEW packages will be installed:
  libnvidia-common-525-server libnvidia-egl-wayland1 libnvidia-gl-525-server
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 198 MB of archives.
After this operation, 457 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 libnvidia-common-525-server all 525.125.06-0ubuntu0.22.04.1 [9,574 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libnvidia-egl-wayland1 amd64 1:1.1.9-1.1 [26.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 libnvidia-gl-525-server amd64 525.125.06-0ubuntu0.22.04.1 [198 MB]
Fetched 198 MB in 11s (17.8 MB/s)
Selecting pre

In [2]:
import numpy as np
import torch
from IPython.display import Audio
import gradio as gr
from typing import Tuple
# import pyttsx3
import re


### Text-2-Speech-SpeechT5
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from transformers import SpeechT5HifiGan
from datasets import load_dataset

### Text-2-Speech-Bark
from transformers import BarkModel, BarkProcessor


### GPT4ALL
import gpt4all

### Speech-2-Text
import whisper


In [3]:
def interview_bot( model_name="orca-mini-7b.ggmlv3.q4_0.bin" , allow_download=True):
  try:
    m = gpt4all.GPT4All(model_name, device='gpu', allow_download=allow_download)
    print('Loaded for GPU')
  except:
    m = gpt4all.GPT4All(model_name, device='cpu',allow_download= allow_download)
    print('Loaded for CPU')

  return m


Text-2-Speech-**SpeechT5**

In [4]:
class Text2_verbal_speech():

  def __init__(self, model_name="microsoft/speecht5_tts" , vocoder_name="microsoft/speecht5_hifigan", dataset_name="Matthijs/cmu-arctic-xvectors",rate=16000 ) -> None:

    self.processor          =   SpeechT5Processor.from_pretrained(model_name)
    self.model              =   SpeechT5ForTextToSpeech.from_pretrained(model_name)
    self.vocoder            =   SpeechT5HifiGan.from_pretrained(vocoder_name)
    self.embeddings_dataset =   load_dataset(dataset_name, split="validation")
    self.rate               =   rate

  def convert(self,response):

    inputs             = self.processor(text=response, return_tensors="pt")
    speaker_embeddings = torch.tensor(self.embeddings_dataset[87]["xvector"]).unsqueeze(0)
    speech             = self.model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=self.vocoder)
    return speech



In [5]:
##
response= "Here's a question for you: What is the difference between supervised and unsupervised learning ? "

def demo_verbal_speech(response,sampling_rate=16000):
  text2speech=Text2_verbal_speech()
  speech=text2speech.convert(response)
  return (sampling_rate , speech.numpy())
# # speech=text2speech.convert(response)

In [6]:
class S2T():
  def __init__(self, model_name="base.en"):

    self.model_name=model_name
    self.model=whisper.load_model(model_name)

  def convert(self, audio):
    text=self.model.transcribe(audio)["text"]
    return text

  def transcribe_through_windows(self,audio):

  # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio("audio.mp3")
    audio = whisper.pad_or_trim(audio)

      # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

      # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

      # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)

      # print the recognized text
    return result.text

In [7]:
# transcriber = S2T()
# transcriber.convert("/content/openai-whisper-webapp/two_cities.mp3")

In [8]:
# x=demo_verbal_speech(response)

In [9]:
# Audio(x[1],x[0])

In [10]:
# demo_t2s_v = gr.Interface(fn = demo_verbal_speech ,
#                      inputs = 'text',
#                      outputs = 'audio',
#                      verbose = True,
#                      title = ' Asking questions from text to Audio ',
#                     #  description = 'A simple application to convert PDF files in audio speech. Upload your own file, or click one of the examples to load them.',
#                      article =
#                         '''<div>
#                             <p style="text-align: center"> Adding Human voice for audio </p>
#                         </div>'''

#                     )

# demo_t2s_v.launch(debug=True)

Text-2-Speech-**BARK**

In [11]:
# @title
class Text2_non_verbal_speech():

  def __init__(self, model_name="suno/bark-small" , speaker_name="v2/en_speaker_1") -> None:

    self.model_name=model_name
    self.speaker_name=speaker_name
    self.model = BarkModel.from_pretrained(model_name).to("cuda")
    self.processor = BarkProcessor.from_pretrained(model_name)

  def convert(self,response):

    inputs = self.processor(response , voice_preset=self.speaker_name).to("cuda")
    speech = self.model.generate(**inputs).cpu().numpy()
    sampling_rate = self.model.generation_config.sample_rate
    return speech,sampling_rate


In [12]:
# @title
def demo_non_verbal_speech(response):
  text2speech_nv=Text2_non_verbal_speech()
  speech,rate =text2speech_nv.convert(response)
  return speech, rate
  # return (sampling_rate , speech.numpy())

In [13]:
# @title
response = ["Hello"]
speech,rate = demo_non_verbal_speech(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


In [ ]:
# @title
demo_t2s_v = gr.Interface(fn = demo_verbal_speech ,
                     inputs = 'text',
                     outputs = 'audio',
                     verbose = True,
                     title = ' Asking questions from text to Audio ',
                    #  description = 'A simple application to convert PDF files in audio speech. Upload your own file, or click one of the examples to load them.',
                     article =
                        '''<div>
                            <p style="text-align: center"> Adding Human voice for audio </p>
                        </div>'''

                    )

demo_t2s_v.launch(debug=True)

<ipython-input-14-ea58a3542100>:2: GradioDeprecationWarning: `verbose` parameter is deprecated, and it has no effect
  demo_t2s_v = gr.Interface(fn = demo_verbal_speech ,


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://50b96a4721bbafbcd0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
# def Text_to_non_verbal_speech(input_list):
#     model_name = "suno/bark-small"
#     model = BarkModel.from_pretrained(model_name).to("cuda")
#     processor = BarkProcessor.from_pretrained(model_name)
#     inputs = processor(input_list, voice_preset="v2/en_speaker_6").to("cuda")
#     speech_output = model.generate(**inputs).cpu().numpy()
#     sampling_rate = model.generation_config.sample_rate

#     return speech_output,sampling_rate


In [ ]:
# input_list = [
#     "[clears throat] Hello Let's start the interview [laughter]",
#     "Can you explain the concept of word embeddings in NLP? Discuss the differences between traditional methods (e.g., one-hot encoding) and distributed word embeddings (e.g., Word2Vec, GloVe). How do pre-trained word embeddings like Word2Vec and FastText benefit NLP tasks?"
# ]

In [ ]:
# def demo_nv_verbal_speech(text):
#     speech_output,sampling_rate  = Text_to_non_verbal_speech(input_list=[text])
#     return ( sampling_rate , speech_output[0])

# # Audio(speech_output[0], rate=sampling_rate)

In [ ]:
# demo_nv_verbal_speech("hello")

In [ ]:
# demo_t2s_v = gr.Interface(fn = demo_nv_verbal_speech ,
#                      inputs = 'text',
#                      outputs = 'audio',
#                      verbose = True,
#                      title = ' Asking questions from text to Audio ',
#                     #  description = 'A simple application to convert PDF files in audio speech. Upload your own file, or click one of the examples to load them.',
#                      article =
#                         '''<div>
#                             <p style="text-align: center"> Adding Human voice for audio </p>
#                         </div>'''

#                     )

# demo_t2s_v.launch(debug=True,share=True)

Mechine like voice text-2-audio

In [ ]:
# @title
class mechanical_speech():
    def __init__(self):

        self.engine = pyttsx3.init()

        self.rate = self.engine.getProperty('rate')   # getting details of current speaking rate
        self.engine.setProperty('rate', 100)     # setting up new voice rate

        self.volume = self.engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
        self.engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1

        self.voices = self.engine.getProperty('voices')       #getting details of current voice
        self.engine.setProperty('voice', self.voices[0].id)   #changing index, changes voices. 1 for female


    def texttoaud(self,text):

        self.engine.say(text)
        self.engine.runAndWait()
        self.engine.stop()

        audio = self.engine.save_to_file(text, 'test.mp3')
        self.engine.runAndWait()
        return audio

In [ ]:
#  mechanical_speech().texttoaud("Hi how was the interview, lets start the interview. Iam going to ask you the first question, you need to answer it (a+b) whole square is equal to what")

In [ ]:
# %%time

# m=interview_bot( model_name="orca-mini-7b.ggmlv3.q4_0.bin" , allow_download=True)

# system_template = ''' You  have to take  interviwe of candidate for job position of data scientist.
#                   Interview topic include deep learning, machine learning, NLP and LLMs. Generated question should be classified in easy , medium and hard level.'''
# # many models use triple hash '###' for keywords, Vicunas are simpler:
# prompt_template = 'USER: {0}\nASSISTANT: '

# with m.chat_session(system_template , prompt_template):
#     response = m.generate(prompt='Generate 3 question. And Label them according to difficulty level ',
#                           temp=1,
#                           max_tokens=1000,
#                           top_k=40,
#                           top_p=0.4,
#                           repeat_penalty=1.18,
#                           repeat_last_n=64,
#                           n_batch=8,
#                           n_predict=None,
#                           streaming=False)
#     print(response)

# response1 = m.generate(prompt='', temp=0)


Fine-tune GPT4ALL

In [ ]:
## Dataset

In [ ]:
# %%time
# system_template = ' If you understood question and answer and diffiulty level given in next paragraph then reply  in one word, "yes" or "no" .'
# # if we want to make custome prompt. Except USER and Assistant.
# # prompt_template = 'Question: {0}\n Answer:{1}\n Difficulty_level: {2} \n Understood: '

# prompt_templat = 'USER: "Question is {0} .And answer is {1}. Difficulty level is {2}". \nASSISTANT:'
# Data=["Is logistic regression a classification algorithm ?","yes","Easy"]


# input=system_template+prompt_templat.format(Data[0],Data[1],Data[2])
# print(input)
# # with m.chat_session(system_template,prompt_template):
# response = m.generate(prompt=input,
#                           temp=1,
#                           max_tokens=1,
#                           top_k=40,
#                           top_p=0.4,
#                           repeat_penalty=1.18,
#                           repeat_last_n=64,
#                           n_batch=8,
#                           n_predict=None,
#                           streaming=False)
# print(response)

# # response1 = m.generate(prompt='', temp=0)


Speech-2-Text

In [ ]:
# obj = S2T()
# print(obj.convert("/content/openai-whisper-webapp/two_cities.mp3"))
# Audio("/content/openai-whisper-webapp/two_cities.mp3")

Analyze the answer given by student.


In [ ]:
# %%time

# system_template = ''' Please review the user's answer on the topic of machine learning and provide a concise list of machine learning-related mistakes, if any, in bullet points. Be specific in pointing out errors, inaccuracies, or areas that require improvement."
#                     '''
# # many models use triple hash '###' for keywords, Vicunas are simpler:
# prompt_template = 'USER: Question is "{0}". Answer is "{1}" \nASSISTANT:'

# data={"question":"What is the difference between supervised and unsupervised learning? " ,
#       "answer" : "Supervised learning is a type of machine learning where the algorithm learns from labeled data.This means that there is already some data available that has been categorized or classified by humans. The algorithm then uses this labeled data to learn patterns and make predictions about new, unlabeled data. For example, if you wanted to teach a computer how to recognize handwritten digits, you would provide the algorithm with a set of labeled images and it would use this information to learn how to identify these patterns in new, unlabeled data.\
# Unsupervised learning, on the other hand, is a type of machine learning where the algorithm learns from unlabeled data. This means that there is no pre-existing labeled data available, and the algorithm must find patterns and structure in the data on its own. For example, if you wanted to teach a computer how to identify different types of animals based on their characteristics (e.g. size, fur color, etc.), you would provide the algorithm with unlabeled images of various animals and it would use this information to identify patterns and similarities between these animals."
# }


# input = system_template + prompt_template.format(data["question"],data["answer"])

# # with m.chat_session(system_template,prompt_template):
# response = m.generate(prompt= input ,
#                       temp=1,
#                       max_tokens=2000,
#                       top_k=40,
#                       top_p=0.4,
#                       repeat_penalty=1.18,
#                       repeat_last_n=64,
#                       n_batch=8,
#                       n_predict=None,
#                       streaming=False)
# print(response)
# # response1 = m.generate(prompt='', temp=0)

In [ ]:
# def count_mistake(report):
#   # Sample paragraph
#   # report = " This is a sample paragraph with numbers like 123, 456, and 789. The last number is 0."

#   # Define a regular expression pattern to match numbers
#   pattern = r'\d+'

#   # Use the findall method to find all numbers in the paragraph
#   numbers = re.findall(pattern, report)

#   # If there are numbers found, get the last one

#   count_mistake = numbers[-1]
#   print("Total number of mistakes: ", count_mistake)


Calculate score:

In [ ]:
class scoring():
  def __init__(self ):

    self.sub_ques_weight=0
    self.mcq_ques_weight=0
    self.penalty_per_mistake=0

  def set_weights( self, difficulty_level):

    if difficulty_level=="easy":
      self.sub_ques_weight=10
      self.mcq_ques_weight=1
      self.penalty_per_mistake=1

    elif difficulty_level=="medium":
      self.sub_ques_weight=20
      self.mcq_ques_weight=2
      self.penalty_per_mistake=2

    elif difficulty_level=="hard":
      self.sub_ques_weight=30
      self.mcq_ques_weight=3
      self.penalty_per_mistake=3

  def calculate_mark_per_question(self,subjective_mistake_count, mcq_mistake_count , difficulty_level ):

    # print("difficulty_levle" , difficulty_level)
    self.set_weights(difficulty_level)

    # print( "assigned_weight" , self.sub_ques_weight , self.mcq_ques_weight , self.penalty_per_mistake )

    # subjective_mistake_count = int(subjective_mistake_count)

    if subjective_mistake_count > 5:
      # print("how it entered here")
      score=0
      return 0,0,0

    else:
      # print(" else loop entered")
      subjective_score = self.sub_ques_weight - (subjective_mistake_count*self.penalty_per_mistake)
      # print(" subjective score" , subjective_score)
      mcq_score        = self.mcq_ques_weight*(3-mcq_mistake_count)
      # print("mcq_score" , mcq_score )

      score = 0.8 * subjective_score + 0.2 *  mcq_score
      # print("score" , score)
    # return score
    return subjective_score, mcq_score ,score   # return this for demo purpose only.

  def evaluate_all_question(self , mark_list):    ###  mark_list is list of dictionary whose keys are subjective_mistake_count, mcq_mistake_count , difficulty_level. ###

    score=0
    for mark in mark_list:

      temp=self.calculate_mark_per_question( mark["subjective_mistake_count"] , mark["mcq_mistake_count"] , mark["difficulty_level"] )
      # print(temp)
      score+=temp

    return score

In [ ]:
# mark_list=[{"subjective_mistake_count": 3,  "mcq_mistake_count" :2 ,"difficulty_level" : "easy" } ]

Demo

In [ ]:
# @title
import gradio as gr


def change_textbox(choice):
    if choice == "short":
        return gr.Textbox.update(lines=2, visible=True)
    elif choice == "long":
        return gr.Textbox.update(lines=8, visible=True)
    else:
        return gr.Textbox.update(visible=False)


with gr.Blocks() as block:
    radio = gr.Radio(
        ["short", "long", "none"], label="What kind of essay would you like to write?"
    )
    text = gr.Textbox(lines=2, interactive=True)

    radio.change(fn=change_textbox, inputs=radio, outputs=text)
    block.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f6e75f149a7e0552c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
demo=gr.Blocks()

with demo:

    gr.Markdown("Let's start the Interview.")

    with gr.Tabs():
        with gr.TabItem("Question Generation"):

          with gr.Row():
            with gr.Column():
              level = gr.Dropdown( ["easy" , "medium", "hard" ], label="Difficulty level", info="choose difficulty level of question.")
              radio1 = gr.Radio(["v1.0" , "v2.0"], label="Version", info="choose version")

            text1  = gr.Textbox( lines=4 )

          b1=gr.Button("Generate")
          b1.click(fn=generate_question , inputs=[level,radio1] , outputs=text1)

        with gr.TabItem("Ask question"):

          with gr.Row():
            with gr.Column():
              text2 =  gr.Textbox( lines=4 )
              b2_1      = gr.Button("Convert")

            with gr.Column():


              gr.Markdown(" Machine voice ")
              audio2_3    = gr.Audio()
              b2_1.click(fn = mechanical_speech , inputs = text2 , outputs = audio2_3 )

              gr.Markdown(" Human voice + non-speeh sound ")
              dropdown2 = gr.Dropdown(["[laughter]" , "[laughs]" , "[sighs]" ,"[music]" , "[gasps]" , "[clears throat]"], ["[clears throat]", "[laughs]"], multiselect=True , label="Non-speech sounds", info="choose any two")
              audio2_1     = gr.Audio()

              b2_1.click(fn = demo_nv_verbal_speech , inputs = [text2 , dropdown2] , outputs = audio2_1 )

              # gr.Markdown(" Human voice ")
              # audio2_2    = gr.Audio()
              # b2_1.click(fn = demo_verbal_speech , inputs = text2 , outputs = audio2_2 )

        with gr.TabItem("Record candidate's Answer"):
          with gr.Row():
            with gr.Column():
              audio3 = gr.Audio(source="microphone" , type= "filepath")
              b3      = gr.Button("Convert")

            text3  = gr.Textbox( lines=4 )

          b3.click(fn=convert_audio_to_text, inputs =audio3 , outputs = text3)

        with gr.TabItem("Generate MCQ based on generated question"):
          with gr.Column():
              radio4 = gr.Radio(["v1.0" , "v2.0"], label="Version", info="choose version")
              b4 = gr.Button("Create MCQ")

              text4_1 = gr.Textbox( lines=4 , label = "Question" , info = "This question is generated in first tab" , show_label=True)
              text4 = gr.Textbox( lines=4 , label = "MCQ" , info = "MCQ is based on mentioned question" )


          b4.click(fn = lambda x: x , inputs = text1 , outputs = text4_1  )
          b4.click(fn = generated_mcq , inputs = [text1 , radio4] , outputs = text4 )


        with gr.TabItem("Find out mistakes"):

          b6 = gr.Button("Analyze candidate's answer")

          text6_3 = gr.Textbox( lines=4, label = "mistakes")

          b6.click(fn = find_mistake , inputs =[text1 , radio1 ] ,outputs= text6_3 )


        with gr.TabItem("calculate score"):
          with gr.Row():
            with gr.Column():

              # radio5 = gr.Radio([0,1,2,3], label="MCQ", info="How many MCQ's are wrong ?")
              text5_1 = gr.Textbox( label=" Total mistakes in subjective question")
              text5_2 = gr.Textbox( label=" Total wrong MCQ")
              level5 = gr.Dropdown( ["easy" , "medium", "hard" ], label="Difficulty level", info="choose difficulty level of question.")
              # text5_3 = gr.Textbox( label=" total number of mistakes in candidate's answer")
              button5 = gr.Button("calculate score")

            with gr.Column():
              text5_5 = gr.Textbox(lines=2 , label="subjective_score")
              text5_3 = gr.Textbox(lines=2 , label="MCQ_score")
              text5_4 = gr.Textbox(lines=2 , label="Total_score")

            button5.click( fn = calculate_score , inputs=[text5_1, text5_2 , level5] ,outputs = [text5_5 , text5_3 , text5_4])


demo.launch(share=True , debug=True)

#  with gr.Column():

In [ ]:
%%time
question_generator_model= interview_bot( model_name="orca-mini-7b.ggmlv3.q4_0.bin" , allow_download=True)


In [ ]:

def generate_question( dificulty_level, version):

  if version == "v1.0":

    # m = interview_bot( model_name="orca-mini-7b.ggmlv3.q4_0.bin" , allow_download=True)
    system_template = ''' You  have to take  interviwe of candidate for job position of data scientist.Interview topic include machine learning concepts.'''
    # many models use triple hash '###' for keywords, Vicunas are simpler:

    prompt_template = 'USER: Generate one  {0} question from the mentioned domain \nASSISTANT: '
    input_prompt=system_template + prompt_template.format( dificulty_level )

    response = question_generator_model.generate(prompt=input_prompt,
                                                  temp=1,
                                                  max_tokens=100,
                                                  top_k=40,
                                                  top_p=0.4,
                                                  repeat_penalty=1.18,
                                                  repeat_last_n=64,
                                                  n_batch=8,
                                                  n_predict=None,
                                                  streaming=False)
    return response

  else:
    Question=[" What is the difference between supervised and unsupervised learning?" , "Can you explain the concept of decision trees?" ,  "How do you handle missing data in machine learning models?"]
    idx=np.random.choice([0,1,2])
    return Question[idx]




In [ ]:
# generate_question("easy","v1.0")

In [ ]:
def Text_to_non_verbal_speech(input_list):
    model_name = "suno/bark-small"
    model = BarkModel.from_pretrained(model_name).to("cuda")
    processor = BarkProcessor.from_pretrained(model_name)
    inputs = processor(input_list, voice_preset="v2/en_speaker_6").to("cuda")
    speech_output = model.generate(**inputs).cpu().numpy()
    sampling_rate = model.generation_config.sample_rate

    return speech_output,sampling_rate

In [ ]:
def demo_nv_verbal_speech(text , non_speech_sound):
    text = str(non_speech_sound[0]) +" "+ text + " " + str(non_speech_sound[1])
    print(text)
    speech_output,sampling_rate  = Text_to_non_verbal_speech(input_list=[text])
    return ( sampling_rate , speech_output[0])


In [ ]:
# my_tuple = demo_nv_verbal_speech("hello world" , ["[clears throat]" , "[laughter]"])
# Audio(my_tuple[1], rate=my_tuple[0])

In [ ]:
def demo_verbal_speech(response,sampling_rate=16000):
  text2speech=Text2_verbal_speech()
  speech=text2speech.convert(response)
  # return sampling_rate,speech
  return (sampling_rate , speech.numpy())
# speech=text2speech.convert(response)

In [ ]:
# response= "Here's a question for you: What is the difference between supervised and unsupervised learning ? "
# samping_rate,speech = demo_verbal_speech(response)

In [ ]:
# Audio(speech , samping_rate)

In [ ]:
def mechanical_speech(response):
  return "/content/machine_voice.mp3"

In [ ]:

def convert_audio_to_text(audio):
  transcriber = S2T()
  text = transcriber.convert(audio)
  return text


In [ ]:
def generated_mcq(question , version):
  if version == "v1.0":

    # m = interview_bot( model_name="orca-mini-7b.ggmlv3.q4_0.bin" , allow_download=True)
    system_template = ''' Generate three multiple-choice questions (MCQs) based on the following machine learning question. Give 4 option and final answer to each question. '''
    # many models use triple hash '###' for keywords, Vicunas are simpler:

    prompt_template = 'USER: Question is  {0}  \nASSISTANT: '
    input_prompt=system_template + prompt_template.format( question )

    response = question_generator_model.generate(prompt=input_prompt,
                                                  temp=1,
                                                  max_tokens=1000,
                                                  top_k=40,
                                                  top_p=0.4,
                                                  repeat_penalty=1.18,
                                                  repeat_last_n=64,
                                                  n_batch=8,
                                                  n_predict=None,
                                                  streaming=False)
    return response

  else:
    if question == " What is the difference between supervised and unsupervised learning?" :
      return '''    MCQ 1:
                    What is a key characteristic of supervised learning?
                    A) The absence of labeled data.
                    B) The use of labeled data for training.
                    C) The requirement for no training data.
                    D) The ability to handle missing data.

                    Correct Answer: B) The use of labeled data for training.

                    MCQ 2:
                    In unsupervised learning, what is the primary objective?
                    A) To make predictions on new, unseen data.
                    B) To learn from labeled examples.
                    C) To discover patterns and structures in data without labeled outcomes.
                    D) To maximize model accuracy.

                    Correct Answer: C) To discover patterns and structures in data without labeled outcomes.

                    MCQ 3:
                    Which of the following is an example of supervised learning?
                    A) Image clustering
                    B) Spam email classification
                    C) Principal Component Analysis (PCA)
                    D) Market basket analysis

                    Correct Answer: B) Spam email classification. '''

    elif  question == "Can you explain the concept of decision trees?" :
      return '''  MCQ 1:
                  In a decision tree, what are the nodes representing?
                  A) Decisions or actions
                  B) Data instances
                  C) Features or attributes
                  D) Probability distributions

                  Correct Answer: C) Features or attributes.

                  MCQ 2:
                  What is the primary purpose of a decision tree's leaves (terminal nodes)?
                  A) To represent splits in the data.
                  B) To store decision rules.
                  C) To contain the final class or outcome labels.
                  D) To calculate information gain.

                  Correct Answer: C) To contain the final class or outcome labels.

                  MCQ 3:
                  How does a decision tree select the best feature for splitting?
                  A) By choosing the feature with the highest information gain or Gini impurity reduction.
                  B) By selecting the feature with the lowest entropy.
                  C) By choosing the feature with the most levels.
                  D) By using random feature selection.

                  Correct Answer: A) By choosing the feature with the highest information gain or Gini impurity reduction.   '''
    else:
      return '''  MCQ 1:
                  Which of the following methods is commonly used for handling missing numerical data?
                  A) Dropping rows with missing values
                  B) Replacing missing values with the median
                  C) Converting missing values to a new category
                  D) Ignoring missing values during training

                  Correct Answer: B) Replacing missing values with the median.

                  MCQ 2:
                  What is imputation in the context of handling missing data?
                  A) Removing all missing data points from the dataset
                  B) Replacing missing values with random values
                  C) Filling in missing values with estimated or calculated values
                  D) Transforming missing values into categorical variables

                  Correct Answer: C) Filling in missing values with estimated or calculated values.

                  MCQ 3:
                  Which of the following techniques is suitable for handling missing categorical data?
                  A) Mean imputation
                  B) Regression imputation
                  C) Mode imputation
                  D) Standardization

                  Correct Answer: C) Mode imputation. '''






In [ ]:
%%time
# generated_mcq(  "Can you explain the concept of decision trees?" , "v1.0")

In [ ]:
myobj1 = scoring()
def calculate_score( subjective_mistake_count , mcq_mistake_count, difficulty_level):
  subjective_mistake_count = int(subjective_mistake_count)
  mcq_mistake_count = int(mcq_mistake_count)
  # print("before class function" , subjective_mistake_count , mcq_mistake_count, difficulty_level)
  # print("hello")
  subjective_score, mcq_score ,score =  myobj1.calculate_mark_per_question( subjective_mistake_count , mcq_mistake_count, difficulty_level)
  # print(  " class function end " , subjective_score, mcq_score ,score)
  return subjective_score, mcq_score ,score

In [ ]:
x=calculate_score("3","2" ,"easy")
x

In [ ]:
def find_mistake( question , version):
  if version == "v1.0":


    system_template = ''' Please review the user's answer on the topic of machine learning and provide a concise list of machine learning-related mistakes, if any, in bullet points. Be specific in pointing out errors, inaccuracies, or areas that require improvement."
                        '''
    # many models use triple hash '###' for keywords, Vicunas are simpler:
    prompt_template = 'USER: Question is "{0}". Answer is "{1}" \nASSISTANT:'

    input = system_template + prompt_template.format(question, answer)

    # with m.chat_session(system_template,prompt_template):
    response = question_generator_model.generate(prompt= input ,
                          temp=1,
                          max_tokens=2000,
                          top_k=40,
                          top_p=0.4,
                          repeat_penalty=1.18,
                          repeat_last_n=64,
                          n_batch=8,
                          n_predict=None,
                          streaming=False)
    return response

  else:
    if question == " What is the difference between supervised and unsupervised learning?" :
      return   '''
        In the first sentence, it states that supervised learning is trained on an "unlabeled" dataset, which is incorrect. It should be "labeled dataset."
        In the second sentence, it says that unsupervised learning involves training on "unlabeled data," which is correct.
        In the fourth sentence, it mentions "supervised learning" again, but it should refer to "unsupervised learning."

         '''

    elif question == "Can you explain the concept of decision trees?" :
      return  '''

      Decision trees are not primarily used for image recognition and natural language processing; they are more commonly employed in classification and regression tasks.
      Decision trees do not start at the leaf nodes; they typically start at the root node and traverse down the branches to reach the leaf nodes for predictions.
      The statement that decision trees aim to maximize impurity or minimize information gain is incorrect; the goal is to minimize impurity or maximize information gain.
      Decision trees can handle both categorical and numerical features, contrary to the claim that they can only handle categorical features effectively.
      The assertion that decision trees are immune to overfitting is incorrect; they are prone to overfitting, and techniques like pruning and using ensemble methods are employed to mitigate this issue.
      '''
    else:
      return '''
      "or reliable results" should be "or unreliable results."
      "not crucial to choose the method" should be "crucial to choose the method."
      "biased or reliable results" should be "biased or reliable results."
       '''


In [ ]:
find_mistake( " What is the difference between supervised and unsupervised learning?" , " " , "v2.0")

(' What is the difference between supervised and unsupervised learning?',
 ' ',
 '  \n        In the first sentence, it states that supervised learning is trained on an "unlabeled" dataset, which is incorrect. It should be "labeled dataset."\n        In the second sentence, it says that unsupervised learning involves training on "unlabeled data," which is correct.\n        In the fourth sentence, it mentions "supervised learning" again, but it should refer to "unsupervised learning."\n\n         ')